In [16]:
import cv2
import pytesseract
import random
import string

def detect_text_change(video_path, save_path):
    cap = cv2.VideoCapture(video_path)
    fg_bg = cv2.createBackgroundSubtractorMOG2()
    text_on = False
    count = 0
    current_text = ""
    out = None
    videos = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        fg_mask = fg_bg.apply(frame)
        white_pixels = cv2.countNonZero(fg_mask[200:, -200:])

        if white_pixels > 0 and not text_on:
            text_on = True
            text = ''
            text += " " + pytesseract.image_to_string(frame[400:, 800:])
            current_text = text.strip()
            # first word
            if out is None:
                if current_text == '':
                    add_text = ''.join(''.join(random.choice(string.ascii_lowercase) for i in range(5)))
                    # add_text = ''.join(random.choice(string.ascii_lowercase))

                    out = cv2.VideoWriter(f"{save_path}/{current_text + add_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))

                else: out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
                # print('out is None')
            else:
                if current_text != prev_text:
                    out.release()
                    if current_text == '':
                        add_text = ''.join(''.join(random.choice(string.ascii_lowercase) for i in range(5)))
                        # add_text = ''.join(random.choice(string.ascii_lowercase))
                        out = cv2.VideoWriter(f"{save_path}/{current_text + add_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))

                    else: out = cv2.VideoWriter(f"{save_path}/{current_text}.avi", cv2.VideoWriter_fourcc(*"MJPG"), 30.0, (frame.shape[1], frame.shape[0]))
                    # print('current_text != prev')
                # print('else')
            prev_text = current_text
            # print(f"Text {current_text} on")
            start_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        if white_pixels == 0 and text_on:
            text_on = False
            out.release()
            # print(f"Text {current_text} off")
            end_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES)) - 1
            if current_text == '':
                videos.append((current_text + add_text, start_frame, end_frame))
            else: videos.append((current_text, start_frame, end_frame))
        
        if text_on:
            out.write(frame)
            
        cv2.imshow('Frame', frame)
        # cv2.imshow('FG Mask', fg_mask)
        # cv2.imshow('200', fg_mask[200:, -200:])
        cv2.imshow('400', fg_mask[400:, 900:])

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return videos



In [17]:
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_10/Processed_data/Video/Subject_10_05.mp4"
save = "C:/Users/men22/Documents/test1/"
stamps = detect_text_change(path, save)
stamps

[('jcdkn', 1, 1),
 ('jcdkn', 31, 37),
 ('jcdkn', 106, 176),
 ('Pen', 200, 222),
 ('kcgjs', 277, 351),
 ('Bigail', 372, 394),
 ('aupac', 468, 585),
 ('aupac', 639, 718),
 ('Desg', 733, 755),
 ('fyknt', 821, 938),
 ('Galw', 987, 1061),
 ('Galw', 1082, 1104),
 ('aiclig', 1185, 1262),
 ('Ffair', 1279, 1301),
 ('Faint', 1365, 1442),
 ('Faint', 1460, 1482),
 ('Theatr', 1549, 1624),
 ('Theatr', 1644, 1666),
 ('snipq', 1725, 1842),
 ('snipq', 1897, 1974),
 ('snipq', 1981, 1983),
 ('snipq', 1992, 2014),
 ('snipq', 2073, 2154),
 ('Llaw', 2168, 2190),
 ('tpodv', 2266, 2383),
 ('=\nU', 2438, 2514),
 ('Chi', 2521, 2521),
 ('Chi', 2533, 2555),
 ('ey\ni)\nc', 2629, 2719),
 ('Het', 2724, 2746),
 ('lrznv', 2802, 2919),
 ('lrznv', 2976, 3052),
 ('Jam', 3071, 3093),
 ('laxci', 3170, 3246),
 ('Rhaw', 3265, 3287),
 ('sXeloleya', 3350, 3424),
 ('sXeloleya', 3445, 3467),
 ('gwgcl', 3527, 3620),
 ('Vaal)', 3622, 3644),
 ('nmcfe', 3698, 3775),
 ('Mawr', 3793, 3815),
 ('Neidr', 3888, 3963),
 ('Neidr', 3983, 400

In [282]:
stamps

[('gmmll', 1, 1),
 ('gmmll', 31, 37),
 ('gmmll', 57, 103),
 ('Pen', 121, 122),
 ('Pen', 151, 173),
 ('Bigail', 228, 305),
 ('Bigail', 323, 345),
 ('qddvo', 419, 504),
 ('qddvo', 514, 536),
 ('Desg', 590, 670),
 ('Desg', 684, 706),
 ('Cath', 772, 848),
 ('Cath', 867, 889),
 ('Galw', 969, 1086),
 ('Ffair', 1145, 1225),
 ('Ffair', 1239, 1261),
 ('Faint', 1305, 1382),
 ('Faint', 1400, 1422),
 ('Theatr', 1511, 1588),
 ('Theatr', 1606, 1628),
 ('Ddoe', 1692, 1784),
 ('DY s fer)', 1787, 1809),
 ('cvrjf', 1871, 1988),
 ('Llaw', 2058, 2148),
 ('Llaw', 2153, 2175),
 ('Siop', 2220, 2298),
 ('Siop', 2315, 2337),
 ('Chi', 2407, 2483),
 ('Chi', 2491, 2491),
 ('Chi', 2502, 2524),
 ('Het', 2577, 2664),
 ('Het', 2672, 2694),
 ('Cwtsh', 2774, 2859),
 ('Cwtsh', 2869, 2891),
 ('Jam', 2951, 3038),
 ('Jam', 3046, 3068),
 ('Rhaw', 3121, 3196),
 ('Rhaw', 3216, 3238),
 ('is Xeloleya', 3311, 3401),
 ('Robot', 3406, 3428),
 ('Vaal)', 3489, 3581),
 ('Vaal)', 3584, 3606),
 ('Mawr', 3649, 3729),
 ('Mawr', 3744, 376

In [22]:
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_9/Processed_data/Video/Subject_09_05.mp4"
save = "C:/Users/men22/Documents/test1/"
stamps1 = detect_text_change(path, save)
stamps1

[('yzuuw', 1, 1),
 ('yzuuw', 31, 37),
 ('yzuuw', 57, 103),
 ('Pen', 121, 122),
 ('Pen', 151, 173),
 ('Bigail', 228, 305),
 ('Bigail', 323, 345),
 ('kxtgn', 419, 504),
 ('kxtgn', 514, 536),
 ('Desg', 590, 670),
 ('Desg', 684, 706),
 ('Cath', 772, 848),
 ('Cath', 867, 889),
 ('Galw', 969, 1086),
 ('Ffair', 1145, 1225),
 ('Ffair', 1239, 1261),
 ('Faint', 1305, 1382),
 ('Faint', 1400, 1422),
 ('Theatr', 1511, 1588),
 ('Theatr', 1606, 1628),
 ('Ddoe', 1692, 1784),
 ('DY s fer)', 1787, 1809),
 ('wncnv', 1871, 1988),
 ('Llaw', 2058, 2148),
 ('Llaw', 2153, 2175),
 ('Siop', 2220, 2298),
 ('Siop', 2315, 2337),
 ('Chi', 2407, 2483),
 ('Chi', 2491, 2491),
 ('Chi', 2502, 2524),
 ('Het', 2577, 2664),
 ('Het', 2672, 2694),
 ('Cwtsh', 2774, 2859),
 ('Cwtsh', 2869, 2891),
 ('Jam', 2951, 3038),
 ('Jam', 3046, 3068),
 ('Rhaw', 3121, 3196),
 ('Rhaw', 3216, 3238),
 ('is Xeloleya', 3311, 3401),
 ('Robot', 3406, 3428),
 ('Vaal)', 3489, 3581),
 ('Vaal)', 3584, 3606),
 ('Mawr', 3649, 3729),
 ('Mawr', 3744, 376

In [71]:
def extract_segments2(video_path, segments, output_path):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    stamps1 = segments

    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None

    for i, segment in enumerate(segments):
        current_text, start_frame, end_frame = segment
        # if end_frame-start_frame<=3:break

        # check if difference between start and end frame is less than 25
        if out is not None and prev_text == current_text:
            # join current segment to previous segment
            out.release()
            out = None
            segments[i] = (current_text, last_start_frame, end_frame)
            if i+1 < len(segments) and segments[i+1][0] == current_text: 
                segments[i] = (current_text, last_start_frame, segments[i+1][2])
                segments.pop(i+1)
            continue


        # extract video segment
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        add_text = ''.join(''.join(random.choice(string.ascii_lowercase) for i in range(6)))
        out = cv2.VideoWriter(f"{output_path}/segment_{current_text + add_text}.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        last_start_frame, last_end_frame, prev_text = start_frame, end_frame, current_text
        out.release()

    cap.release()
    return stamps1

In [21]:
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_05.mp4"
save = "C:/Users/men22/Documents/test/"
segemnts = extract_segments2(path, stamps, save)
segemnts

[('jcdkn', 1, 1),
 ('jcdkn', 1, 176),
 ('Pen', 200, 222),
 ('kcgjs', 277, 351),
 ('Bigail', 372, 394),
 ('aupac', 468, 585),
 ('aupac', 468, 718),
 ('Desg', 733, 755),
 ('fyknt', 821, 938),
 ('Galw', 987, 1061),
 ('Galw', 987, 1104),
 ('aiclig', 1185, 1262),
 ('Ffair', 1279, 1301),
 ('Faint', 1365, 1442),
 ('Faint', 1365, 1482),
 ('Theatr', 1549, 1624),
 ('Theatr', 1549, 1666),
 ('snipq', 1725, 1842),
 ('snipq', 1725, 2154),
 ('Llaw', 2168, 2190),
 ('tpodv', 2266, 2383),
 ('=\nU', 2438, 2514),
 ('Chi', 2521, 2521),
 ('Chi', 2521, 2555),
 ('ey\ni)\nc', 2629, 2719),
 ('Het', 2724, 2746),
 ('lrznv', 2802, 2919),
 ('lrznv', 2802, 3052),
 ('Jam', 3071, 3093),
 ('laxci', 3170, 3246),
 ('Rhaw', 3265, 3287),
 ('sXeloleya', 3350, 3424),
 ('sXeloleya', 3350, 3467),
 ('gwgcl', 3527, 3620),
 ('Vaal)', 3622, 3644),
 ('nmcfe', 3698, 3775),
 ('Mawr', 3793, 3815),
 ('Neidr', 3888, 3963),
 ('Neidr', 3888, 4005),
 ('Cyngor', 4070, 4162),
 ('Cyngor', 4070, 4187),
 ('ldnok', 4252, 4344),
 ('laith', 4347, 

In [23]:
import glob

# function to extract the paths for files froma path
def load_paths(data_path):
    files = []
    files.append(glob.glob(data_path, 
                recursive = True))
    return files[0]

In [24]:
import os

path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_05.mp4"
print("file exists?", os.path.exists(path))

file exists? True


In [29]:
data_path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/*text/Participant_*"
files = load_paths(data_path + "/Processed_data/Video/Subject_0*_0*.mp4")

In [41]:
(files)

['C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_3_no_labels\\Processed_data\\Video\\Subject_03_09.mp4',
 'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_4_no_labels\\Processed_data\\Video\\Subject_04_09.mp4',
 'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_5_no_labels\\Processed_data\\Video\\Subject_05_09.mp4',
 'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\text\\Participant_6\\Processed_data\\Video\\Subject_06_09.mp4',
 'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\text\\Participant_7\\Processed_data\\Video\\Subject_07_09.mp4',
 'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\text\\Participant_9\\Processed_data\\Video\\Subject_09_09.mp4']

Extracting all path files for each video (multiple participants)

In [46]:
data_path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/*text/Participant_*"

paths = []
for i in range(1,10):
    print(i)
    files = load_paths(data_path + "/Processed_data/Video/Subject_*_0" + str(i) + ".mp4")
    paths.append(files)

1
2
3
4
5
6
7
8
9


In [48]:
paths

[['C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_12_no_labels\\Processed_data\\Video\\Subject_12_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_13_no_labels\\Processed_data\\Video\\Subject_13_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_3_no_labels\\Processed_data\\Video\\Subject_03_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_4_no_labels\\Processed_data\\Video\\Subject_04_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\notext\\Participant_5_no_labels\\Processed_data\\Video\\Subject_05_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\text\\Participant_10\\Processed_data\\Video\\Subject_10_01.mp4',
  'C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\\text\\Participant_6\\Processed_data\\Video\\Subject_06_01.mp4',
  'C:/Users/m

In [58]:
save = "C:/Users/men22/Documents/All_videos/"
segments = []
for i in range(0,9):
    print(i)
    print(paths[i][-1])
    segment_1 = detect_text_change(paths[i][-1], save)
    segments.append(segment_1)
segments

0
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_01.mp4
1
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_02.mp4
2
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_03.mp4
3
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_04.mp4
4
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_05.mp4
5
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_06.mp4
6
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_07.mp4
7
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_9\Processed_data\Video\Subject_09_08.mp4
8
C:/Use

[[('rxjbd', 1, 1),
  ('rxjbd', 31, 37),
  ('A-ch-a', 74, 115),
  ('A-ch-a', 121, 140),
  ('A-ch-a', 168, 190),
  ('A-dd-a', 194, 216),
  ('PAT [e Br |', 289, 311),
  ('A-ff-a', 315, 387),
  ('A-ff-a', 410, 432),
  ('A-ng-a', 470, 533),
  ('A-ng-a', 565, 587),
  ('A-ll-a', 665, 690),
  ('A-ll-a', 760, 782),
  ('A-ph-a', 851, 873),
  ('A-ph-a', 946, 968),
  ('rxtax', 1021, 1085),
  ('rxtax', 1116, 1138),
  ('rxtax', 1200, 1273),
  ('rxtax', 1295, 1317),
  ('A-rh-a', 1393, 1440),
  ('A-rh-a', 1488, 1510),
  ('A-th-a', 1567, 1590),
  ('A-th-a', 1662, 1684)],
 [('yrefr', 1, 1),
  ('yrefr', 31, 37),
  ('yrefr', 119, 185),
  ('yrefr', 213, 235),
  ('Dau', 290, 366),
  ('Dau', 385, 407),
  ('tnfkf', 482, 556),
  ('Tri', 577, 599),
  ('Pedwar', 674, 758),
  ('Pedwar', 768, 790),
  ('ad Laat 2)', 836, 915),
  ('ad Laat #)', 931, 953),
  ('Chwech', 1021, 1100),
  ('Chwech', 1111, 1111),
  ('Chwech', 1116, 1138),
  ('Saith', 1210, 1292),
  ('Saith', 1304, 1326),
  ('Wyth', 1393, 1453),
  ('Wyth', 

In [63]:
segments[0]


[('rxjbd', 1, 1),
 ('rxjbd', 31, 37),
 ('A-ch-a', 74, 115),
 ('A-ch-a', 121, 140),
 ('A-ch-a', 168, 190),
 ('A-dd-a', 194, 216),
 ('PAT [e Br |', 289, 311),
 ('A-ff-a', 315, 387),
 ('A-ff-a', 410, 432),
 ('A-ng-a', 470, 533),
 ('A-ng-a', 565, 587),
 ('A-ll-a', 665, 690),
 ('A-ll-a', 760, 782),
 ('A-ph-a', 851, 873),
 ('A-ph-a', 946, 968),
 ('rxtax', 1021, 1085),
 ('rxtax', 1116, 1138),
 ('rxtax', 1200, 1273),
 ('rxtax', 1295, 1317),
 ('A-rh-a', 1393, 1440),
 ('A-rh-a', 1488, 1510),
 ('A-th-a', 1567, 1590),
 ('A-th-a', 1662, 1684)]

In [74]:
save = "C:/Users/men22/Documents/VIDEOS_ALL/"
new_segments = []
for i, seg in enumerate(segments):
    for a in range(len(paths[i])):
        print(paths[i][a])
        new_seg = extract_segments2(paths[i][a], seg, save)
        print(new_seg)
        new_segments.append(new_seg)
    
new_segments

C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_12_no_labels\Processed_data\Video\Subject_12_01.mp4
[('rxjbd', 1, 1), ('rxjbd', 1, 37), ('A-ch-a', 74, 115), ('A-ch-a', 74, 190), ('A-dd-a', 194, 216), ('PAT [e Br |', 289, 311), ('A-ff-a', 315, 387), ('A-ff-a', 315, 432), ('A-ng-a', 470, 533), ('A-ng-a', 470, 587), ('A-ll-a', 665, 690), ('A-ll-a', 665, 782), ('A-ph-a', 851, 873), ('A-ph-a', 851, 968), ('rxtax', 1021, 1085), ('rxtax', 1021, 1317), ('A-rh-a', 1393, 1440), ('A-rh-a', 1393, 1510), ('A-th-a', 1567, 1590), ('A-th-a', 1567, 1684)]
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_13_no_labels\Processed_data\Video\Subject_13_01.mp4
[('rxjbd', 1, 1), ('rxjbd', 1, 37), ('A-ch-a', 74, 115), ('A-ch-a', 74, 190), ('A-dd-a', 194, 216), ('PAT [e Br |', 289, 311), ('A-ff-a', 315, 387), ('A-ff-a', 315, 432), ('A-ng-a', 470, 533), ('A-ng-a', 470, 587), ('A-ll-a', 665, 690), ('A-ll-a', 665, 782), ('A-ph-a', 851, 873), ('A-

[[('rxjbd', 1, 1),
  ('rxjbd', 1, 37),
  ('A-ch-a', 74, 115),
  ('A-ch-a', 74, 190),
  ('A-dd-a', 194, 216),
  ('PAT [e Br |', 289, 311),
  ('A-ff-a', 315, 387),
  ('A-ff-a', 315, 432),
  ('A-ng-a', 470, 533),
  ('A-ng-a', 470, 587),
  ('A-ll-a', 665, 690),
  ('A-ll-a', 665, 782),
  ('A-ph-a', 851, 873),
  ('A-ph-a', 851, 968),
  ('rxtax', 1021, 1085),
  ('rxtax', 1021, 1317),
  ('A-rh-a', 1393, 1440),
  ('A-rh-a', 1393, 1510),
  ('A-th-a', 1567, 1590),
  ('A-th-a', 1567, 1684)],
 [('rxjbd', 1, 1),
  ('rxjbd', 1, 37),
  ('A-ch-a', 74, 115),
  ('A-ch-a', 74, 190),
  ('A-dd-a', 194, 216),
  ('PAT [e Br |', 289, 311),
  ('A-ff-a', 315, 387),
  ('A-ff-a', 315, 432),
  ('A-ng-a', 470, 533),
  ('A-ng-a', 470, 587),
  ('A-ll-a', 665, 690),
  ('A-ll-a', 665, 782),
  ('A-ph-a', 851, 873),
  ('A-ph-a', 851, 968),
  ('rxtax', 1021, 1085),
  ('rxtax', 1021, 1317),
  ('A-rh-a', 1393, 1440),
  ('A-rh-a', 1393, 1510),
  ('A-th-a', 1567, 1590),
  ('A-th-a', 1567, 1684)],
 [('rxjbd', 1, 1),
  ('rxjbd',

In [182]:
segments2 = [(segemnts[a][0], segemnts[a][1]+10, segemnts[a][2]+20) for a in range(len(segemnts)-1)]

In [69]:
for i in range(len(paths[0])): print(paths[0][i]), print(i)

C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_12_no_labels\Processed_data\Video\Subject_12_01.mp4
0
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_13_no_labels\Processed_data\Video\Subject_13_01.mp4
1
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_3_no_labels\Processed_data\Video\Subject_03_01.mp4
2
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_4_no_labels\Processed_data\Video\Subject_04_01.mp4
3
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\notext\Participant_5_no_labels\Processed_data\Video\Subject_05_01.mp4
4
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_10\Processed_data\Video\Subject_10_01.mp4
5
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\Participant_6\Processed_data\Video\Subject_06_01.mp4
6
C:/Users/men22/OneDrive - University of Sussex/FYP/Participants\text\

In [289]:
import cv2
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

def detect_voices(video_path, audio_path, output_path):
    # load audio file
    audio = AudioSegment.from_wav(audio_path)
    audio_duration = len(audio)
    print(f"Audio duration: {audio_duration} ms")
    
    # define settings for voice detection
    min_silence_len = 100
    silence_thresh = -45
    
    # load video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # calculate the length of each frame in ms
    frame_duration = 1000 / fps
    
    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    
    # iterate over the audio segments
    for i, audio_segment in enumerate(audio[::min_silence_len]):
        start_time = i * min_silence_len
        end_time = start_time + min_silence_len
        if end_time > audio_duration:
            end_time = audio_duration
        
        # detect if the audio segment contains voice
        if audio_segment.dBFS > silence_thresh:
            start_frame = int(start_time / frame_duration)
            end_frame = int(end_time / frame_duration)
            
            # extract video segment
            cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
            out = cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, 30, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
            for frame_idx in range(start_frame, end_frame+1):
                ret, frame = cap.read()
                if not ret:
                    break
                out.write(frame)
                    
            out.release()
            
    cap.release()

In [290]:
save = "C:/Users/men22/Documents/test1/"
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_10/Processed_data/Video/Subject_10_02.mp4"
audio = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_10/Processed_data/Audio/Subject_10_02.wav"
segemnts = detect_voices(path, audio, save)

Audio duration: 33201 ms


In [272]:
def detect_voices(video_path, audio_path, output_path):
    # load audio file
    audio = AudioSegment.from_wav(audio_path)
    audio_duration = len(audio)
    print(f"Audio duration: {audio_duration} ms")
    
    # define settings for voice detection
    min_silence_len = 1500
    silence_thresh = -45
    
    # load video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # calculate the length of each frame in ms
    frame_duration = 1000 / fps
    
    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    
    # iterate over the audio segments
    for i, audio_segment in enumerate(audio[::min_silence_len]):
        start_time = i * min_silence_len
        end_time = start_time + min_silence_len
        if end_time > audio_duration:
            end_time = audio_duration
        
        # detect if the audio segment contains voice
        if audio_segment.dBFS > silence_thresh:
            # extend the audio segment by 100ms before and after
            extended_start_time = max(0, start_time - 100)
            extended_end_time = min(end_time + 100, audio_duration)
            extended_audio_segment = audio[extended_start_time:extended_end_time]
            
            # calculate the average loudness of the extended audio segment
            average_loudness = extended_audio_segment.dBFS
            
            # if the average loudness is higher than the silence threshold, extract the video segment
            if average_loudness > silence_thresh:
                start_frame = int(start_time / frame_duration)
                end_frame = int(end_time / frame_duration)
                
                # extract video segment
                cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
                out = cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
                for frame_idx in range(start_frame, end_frame+1):
                    ret, frame = cap.read()
                    if not ret:
                        break
                    out.write(frame)
                        
                out.release()
            
    cap.release()

In [273]:
save = "C:/Users/men22/Documents/from_audio/"
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_05.mp4"
audio = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Audio/Subject_06_05.wav"
segemnts = detect_voices(path, audio, save)

Audio duration: 78747 ms


In [327]:
import cv2
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

def detect_voices(video_path, audio_path, output_path):
    # load audio file
    audio = AudioSegment.from_wav(audio_path)
    audio_duration = len(audio)
    print(f"Audio duration: {audio_duration} ms")
    
    # define settings for voice detection
    min_silence_len = 100
    silence_thresh = -45
    
    # load video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # calculate the length of each frame in ms
    frame_duration = 1000 / fps
    print(f'Frame duration: {frame_duration}')
    
    # define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = None
    video_frames = []
    
    # iterate over the audio segments
    for i, audio_segment in enumerate(audio[::min_silence_len]):
        start_time = i * min_silence_len
        end_time = start_time + min_silence_len
        if end_time > audio_duration:
            end_time = audio_duration
        
        # detect if the audio segment contains voice
        if audio_segment.dBFS > silence_thresh:
            start_frame = int(start_time / frame_duration)
            end_frame = int(end_time / frame_duration)
            video_frames.append((start_frame, end_frame))

    i = 0
    while i<len(video_frames)-1:
        start, end = video_frames[i]
        start1, end1 = video_frames[i+1]
        if start1 - end <= 20:
            video_frames[i] = (start, end1)
            video_frames.pop(i+1)
        else:    
            i+=1




    for i, (start_frame, end_frame) in enumerate(video_frames):
        # start_frame = int(start_time / frame_duration)
        # end_frame = int(end_time / frame_duration)
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        out = cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, fps , (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))
        print(start_frame, end_frame, f'segment_{i}.mp4')
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)
                
        out.release()

    print(video_frames)
    print([e-s for s,e in video_frames]) 
            
    cap.release()

In [328]:
save = "C:/Users/men22/Documents/test1/"
path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_02.mp4"
audio = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Audio/Subject_06_02.wav"
segemnts = detect_voices(path, audio, save)

Audio duration: 33432 ms
59.94005994005994
Frame duration: 16.683333333333334
167 185 segment_0.mp4
353 377 segment_1.mp4
521 539 segment_2.mp4
701 713 segment_3.mp4
893 923 segment_4.mp4
1066 1078 segment_5.mp4
1240 1264 segment_6.mp4
1426 1456 segment_7.mp4
1606 1618 segment_8.mp4
1780 1798 segment_9.mp4
[(167, 185), (353, 377), (521, 539), (701, 713), (893, 923), (1066, 1078), (1240, 1264), (1426, 1456), (1606, 1618), (1780, 1798)]
[18, 24, 18, 12, 30, 12, 24, 30, 12, 18]


In [26]:
import cv2
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

save = "C:/Users/Misha/Documents/test/"
# path = "C:/Users/men22/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_02.mp4"
audio = "C:/Users/Misha/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Audio/Subject_06_02.wav"

sound = AudioSegment.from_file(audio)

# spliting audio files
audio_chunks = split_on_silence(sound, min_silence_len=50, silence_thresh=-42 )

#loop is used to iterate over the output list
for i, chunk in enumerate(audio_chunks):
   output_file = save + f"segment_{i}.mp4"
   print("Exporting file", output_file)
   chunk.export(output_file, format="mp4")

print(type(chunk))
print(dir(chunk))
print()

Exporting file C:/Users/Misha/Documents/test/segment_0.mp4
Exporting file C:/Users/Misha/Documents/test/segment_1.mp4
Exporting file C:/Users/Misha/Documents/test/segment_2.mp4
Exporting file C:/Users/Misha/Documents/test/segment_3.mp4
Exporting file C:/Users/Misha/Documents/test/segment_4.mp4
Exporting file C:/Users/Misha/Documents/test/segment_5.mp4
Exporting file C:/Users/Misha/Documents/test/segment_6.mp4
Exporting file C:/Users/Misha/Documents/test/segment_7.mp4
Exporting file C:/Users/Misha/Documents/test/segment_8.mp4
Exporting file C:/Users/Misha/Documents/test/segment_9.mp4
<class 'pydub.audio_segment.AudioSegment'>
['DEFAULT_CODECS', '__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__seta

In [24]:
import cv2
from pydub import AudioSegment
from pydub.silence import split_on_silence

def extract_video_from_audio2(video_path, audio_path, output_path):
    # Load audio file
    sound = AudioSegment.from_file(audio_path)

    # Split audio file into chunks based on silence
    audio_chunks = split_on_silence(sound, min_silence_len=50, silence_thresh=-42)

    # Load video file
    cap = cv2.VideoCapture(video_path)
    if cap is None: print('error')
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the length of each frame in ms
    frame_duration = 1000 / fps

    # Define video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Loop through audio chunks and extract video segments containing the audio
    for i, chunk in enumerate(audio_chunks):
        start_time = chunk.time_seconds * 1000
        end_time = start_time + len(chunk)

        start_frame = int(start_time / frame_duration)
        end_frame = int(end_time / frame_duration)

        # Set the video position to the start frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Create a video writer for the current segment
        out = cv2.VideoWriter(f"{output_path}/segment_{i}.mp4", fourcc, fps, (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))))

        # Loop through the frames in the segment and write them to the output video
        for frame_idx in range(start_frame, end_frame+1):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()

    cap.release()

In [25]:
save = "C:/Users/Misha/Documents/test/"
path = "C:/Users/Misha/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_02.mp4"
audio = "C:/Users/Misha/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Audio/Subject_06_02.wav"

extract_video_from_audio2(path, audio, save)

59.94005994005994


NameError: name 'start_time' is not defined

In [32]:
chunk  = audio_chunks[6]

In [34]:
min_vol = -42 # adjust this value as needed
start_time = None
end_time = None

# loop through each frame of the segment
for i, frame in enumerate(chunk):
    # if the frame's volume exceeds the minimum volume
    if frame.dBFS >= min_vol:
        # if this is the first frame above the threshold, mark it as the start time
        if start_time is None:
            start_time = i * chunk.frame_width / chunk.frame_rate
        # if we've already marked the start time, keep updating the end time
        end_time = i * chunk.frame_width / chunk.frame_rate

# if we never found a frame above the threshold, assume the whole segment is silence
if start_time is None:
    start_time = 0
    end_time = chunk.duration_seconds

print("Start time:", start_time)
print("End time:", end_time)

Start time: 0.006802721088435374
End time: 0.03600907029478458


In [1]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence

def extract_video_from_audio(audio_file_path, video_file_path, output_directory):
    sound = AudioSegment.from_file(audio_file_path)
    video_name = os.path.splitext(os.path.basename(video_file_path))[0]

    # Split audio into words
    words = split_on_silence(sound, min_silence_len=50, silence_thresh=-42)

    # Extract videos for each word
    for i, word in enumerate(words):
        # Get start and end frame of word
        start_frame = sound.index(word) / sound.frame_width
        end_frame = start_frame + word.duration_seconds * sound.frame_rate

        # Extract video for word
        word_video = f"{output_directory}/word{i}.mp4"
        os.system(f"ffmpeg -i {video_file_path} -ss {start_frame} -to {end_frame} -c:v libx264 -crf 17 -c:a aac -b:a 128k {word_video}")

    print(f"All {len(words)} videos extracted successfully.")


In [3]:
save = "C:/Users/Misha/Documents/test/"
path = "C:/Users/Misha/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Video/Subject_06_02.mp4"
audio = "C:/Users/Misha/OneDrive - University of Sussex/FYP/Participants/text/Participant_6/Processed_data/Audio/Subject_06_02.wav"

extract_video_from_audio(audio, path, save)

AttributeError: 'AudioSegment' object has no attribute 'index'